<img src="img/saturn_logo.png" width="300" />

# Introduction to Dask

Before we get into too much complexity, let's talk about the essentials of Dask.

## What is Dask?

Dask is an open-source framework that enables parallelization of Python code. This can be applied to all kinds of Python use cases, not just machine learning. Dask is designed to work well on single-machine setups and on multi-machine clusters. You can use Dask with pandas, NumPy, scikit-learn, and other Python libraries - for our purposes, we'll focus on how you might use it with PyTorch. If you want to learn more about the other areas where Dask can be useful, there's a [great website explaining all of that](https://dask.org/).

## Why Parallelize?

For our use case, there are a couple of areas where Dask parallelization might be useful for making our work faster and better.
* Loading and handling large datasets (especially if they are too large to hold in memory)
* Running time or computation heavy tasks at the same time, quickly


## Delaying Tasks

Delaying a task with Dask can queue up a set of transformations or calculations so that it's ready to run later, in parallel. This is what's known as "lazy" evaluation - it won't evaluate the requested computations until explicitly told to. This differs from other kinds of functions, which compute instantly upon being called. Many very common and handy functions are ported to be native in Dask, which means they will be lazy (delayed computation) without you ever having to even ask. 

However, sometimes you will have complicated custom code that is written in pandas, scikit-learn, or even base python, that isn't natively available in Dask. Other times, you may just not have the time or energy to refactor your code into Dask, if edits are needed to take advantage of native Dask elements.
If this is the case, you can decorate your functions with `@dask.delayed`, which will manually establish that the function should be lazy, and not evaluate until you tell it. You'd tell it with the processes `.compute()` or `.persist()`, described in the next section. We'll use `@dask.delayed` several times in this workshop to make PyTorch tasks easily parallelized.

### Example 1

In [ ]:
def exponent(x, y):
    '''Define a basic function.'''
    return x ** y

# Function returns result immediately when called
exponent(4, 5)

In [ ]:
import dask

@dask.delayed
def lazy_exponent(x, y):
    '''Define a lazily evaluating function'''
    return x ** y

# Function returns a delayed object, not computation
lazy_exponent(4, 5)

In [ ]:
# This will now return the computation
lazy_exponent(4,5).compute()

### Example 2

We can take this knowledge and expand it - because our lazy function returns an object, we can assign it and then chain it together in different ways later.

Here we return a delayed value from the first function, and call it x. Then we pass x to the function a second time, and call it y. Finally, we multiply x and y to produce z.

In [ ]:
x = lazy_exponent(4, 5)
y = lazy_exponent(x, 2)
z = x * y
z

In [ ]:
z.visualize(rankdir="LR")

In [ ]:
z.compute()

***

## Persist vs Compute

How should we instruct our computer to run the computations we have queued up lazily? We have two choices: `.persist()` and `.compute()`.

First, remember we have several machines working for us right now. We have our Jupyter instance right here running on one, and then our cluster of worker machines also.

### Compute
If we use `.compute()`, we are asking Dask to take all the computations and adjustments to the data that we have queued up, and run them, and bring it all to the surface here, in Jupyter.

That means if it was distributed we want to convert it into a local object here and now. If it's a Dask Dataframe, when we call `.compute()`, we're saying "Run the transformations we've queued, and convert this into a pandas dataframe immediately."

### Persist
If we use `.persist()`, we are asking Dask to take all the computations and adjustments to the data that we have queued up, and run them, but then the object is going to remain distributed and will live on the cluster, not on the Jupyter instance.

So when we do this with a Dask Dataframe, we are telling our cluster "Run the transformations we've queued, and leave this as a distributed Dask Dataframe."

So, if you want to process all the delayed tasks you've applied to a Dask object, either of these methods will do it. The difference is where your object will live at the end.

***

### Example: Distributed Data Objects

When we use a Dask Dataframe object, we can see the effect of `.persist()` and `.compute()` in practice.

In [ ]:
import dask
import dask.dataframe as dd
df = dask.datasets.timeseries()
df.npartitions

So our Dask Dataframe has 30 partitions. So, if we run some computations on this dataframe, we still have an object that has a number of partitions attribute, and we can check it. We'll filter it, then do some summary statistics with a groupby.

In [ ]:
df2 = df[df.y > 0]
df3 = df2.groupby('name').x.std()
print(type(df3))
df3.npartitions

Now, we have reduced the object down to a Series, rather than a dataframe, so it changes the partition number.

We can `repartition` the Series, if we want to!

In [ ]:
df4 = df3.repartition(npartitions=3)
df4.npartitions

What will happen if we use `.persist()` or `.compute()` on these objects?

As we can see below, `df4` is a Dask Series with 161 queued tasks and 3 partitions. We can run our two different computation commands on the same object and see the different results.

In [ ]:
df4

In [ ]:
%%time

df4.persist()

So, what changed when we ran .persist()? Notice that we went from 161 tasks at the bottom of the screen, to just 3. That indicates that there's one task for each partition.

Now, let's try .compute().

In [ ]:
%%time
df4.compute().head()

We get back a pandas Series, not a Dask object at all.

***

## Submit to Cluster

To make this all work in a distributed fashion, we need to understand how we send instructions to our cluster. When we use the `@dask.delayed` decorator, we queue up some work and put it in a list, ready to be run. So how do we send it to the workers and explain what we want them to do?

We use the `distributed` module from Dask to make this work. We connect to our cluster (as you saw in [Notebook 1](01-getting-started.ipynb)), and then we'll use some commands to send instructions.

In [ ]:
from dask_saturn import SaturnCluster
from dask.distributed import Client

cluster = SaturnCluster(
    n_workers = 3, 
    scheduler_size = 'medium', 
    worker_size = 'g4dnxlarge',
    nthreads = 12
)
client = Client(cluster)

In [ ]:
from dask_saturn.core import describe_sizes
describe_sizes()

## Sending Tasks

Now we have created the object `client`. This is the handle we'll use to connect with our cluster, for whatever commands we want to send! We will use a few processes to do this communication: `.submit()` and `.map()`.

* `.submit()` lets us send one task to the cluster, to be run once on whatever worker is free.
* `.map()` lets us send lots of tasks, which will be disseminated to workers in the most efficient way.

There's also `.run()` which you can use to send one task to EVERY worker on the cluster simultaneously. This is only used for small utility tasks, however - like installing a library or collecting diagnostics.

### map Example

For example, you can use `.map()` in this way:

`futures = client.map(function, list_of_inputs)`

This takes our function, maps it over all the inputs, and then these tasks are distributed to the cluster workers. Note: they still won't actually compute yet!

Let's try an example. Recall our `lazy_exponent` function from earlier. We are going to alter it so that it accepts its inputs as a single list, then we can use it with `.map()`.

In [ ]:
@dask.delayed
def lazy_exponent(args):
    x,y = args
    '''Define a lazily evaluating function'''
    return x ** y

In [ ]:
inputs = [[1,2], [3,4], [5,6]]

example_future = client.map(lazy_exponent, inputs)

***

## Processing Results
We have one more step before we use .compute(), which is .gather(). This creates one more instruction to be included in this big delayed job we're establishing: retrieving the results from all of our jobs. It's going to sit tight as well until we finally say .compute().

### gather Example¶

In [ ]:
futures_gathered = client.gather(example_future)

It may help to think of all the work as instructions in a list. We have so far told our cluster: "map our delayed function over this list of inputs, and pass the resulting tasks to the workers", "Gather up the results of those tasks, and bring them back". But the one thing we haven't said is "Ok, now begin to process all these instructions"! That's what `.compute()` will do. For us this looks like:

In [ ]:
futures_computed = client.compute(futures_gathered, sync=False)

We can investigate the results, and use a small list comprehension to return them for later use.

In [ ]:
futures_computed

In [ ]:
futures_computed[0].result()

In [ ]:
results = [x.result() for x in futures_computed]
results

Now we have the background knowledge we need to move on to [Notebook 3](03-single-inference.ipynb).

### Helpful reference links: 
* https://distributed.dask.org/en/latest/client.html
* https://distributed.dask.org/en/latest/manage-computation.html
